# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


In [ ]:
df_first = pd.read_csv('/datasets/geo_data_0.csv')
df_second = pd.read_csv('/datasets/geo_data_1.csv')
df_third = pd.read_csv('/datasets/geo_data_2.csv')

In [ ]:
df_first.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [ ]:
df_second.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [ ]:
df_third.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [ ]:
df_first.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
df_second.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
df_third.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


*Проверяем данные на пропуски и дубликаты*

In [ ]:
df_first.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [ ]:
df_second.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [ ]:
df_third.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [ ]:
df_first.duplicated().sum()

0

In [ ]:
df_second.duplicated().sum()

0

In [ ]:
df_third.duplicated().sum()

0

В данных нет пропусков и дублей. Далее удаляем столбец id,во всех датафреймах он имеет не подходящий тип данных и не нужен для обучения модели

In [ ]:
df_first = df_first.drop(['id'],axis = 1)
df_second = df_second.drop(['id'],axis = 1)
df_third = df_third.drop(['id'],axis = 1)


***Данные не имеют пропуской и дублей.Все данные имеют нужный тип(float). Для обучения модели принято решение удалить столбец id***

## Обучение и проверка модели

*Разобьем данные на обучающую и валидационную выборки в соотношении 75:25.*

In [ ]:
fuatures_first = df_first.drop(['product'],axis = 1)#Выделяем признаки для выборок
fuatures_second = df_second.drop(['product'],axis = 1)
fuatures_third = df_third.drop(['product'],axis = 1)

In [ ]:
target_first = df_first['product']#Выделяем целевые признаки для выборок
target_second= df_second['product']
target_third = df_third['product']

In [ ]:
fuatures_firs_train,fuatures_first_valid,target_first_train,target_first_valid = train_test_split(
fuatures_first,target_first, test_size = 0.25,random_state = 12345)

In [ ]:
fuatures_second_train,fuatures_second_valid,target_second_train,target_second_valid = train_test_split(
fuatures_second,target_second, test_size = 0.25,random_state = 12345)

In [ ]:
fuatures_third_train,fuatures_third_valid,target_third_train,target_third_valid = train_test_split(
fuatures_third,target_third, test_size = 0.25,random_state = 12345)

*Обучим модель с помощью линейной регрессии и сделаем предсказания на валидационной выборке. Посмотрим метрику RMSE и средний объем предсказанного сырья*

In [ ]:
model_first = LinearRegression()#Предсказания для первого региона
model_first.fit(fuatures_firs_train,target_first_train)
predicted_valid_first = model_first.predict(fuatures_first_valid)
mse_first = mean_squared_error(target_first_valid,predicted_valid_first)
print('RMSE=',mse_first**0.5)
print('Средний запас предсказанного сырья ',predicted_valid_first.mean())

RMSE= 37.5794217150813
Средний запас предсказанного сырья  92.59256778438035


In [ ]:
model_second = LinearRegression()#Предсказания для второго региона
model_second.fit(fuatures_second_train,target_second_train)
predicted_valid_second = model_second.predict(fuatures_second_valid)
mse_second = mean_squared_error(target_second_valid,predicted_valid_second)
print('RMSE=',mse_second**0.5)
print('Средний запас предсказанного сырья ',predicted_valid_second.mean())


RMSE= 0.893099286775617
Средний запас предсказанного сырья  68.728546895446


In [ ]:
model_third = LinearRegression()#Предсказания для третьего региона
model_third.fit(fuatures_third_train,target_third_train)
predicted_valid_third = model_third.predict(fuatures_third_valid)
mse_third = mean_squared_error(target_third_valid,predicted_valid_third)
print('RMSE=',mse_third**0.5)
print('Средний запас предсказанного сырья ',predicted_valid_third.mean())

RMSE= 40.02970873393434
Средний запас предсказанного сырья  94.96504596800489


*Проверим модели на адекватность. За основу провурки берем константную модель, которая предсказывает средние значения*

In [ ]:
predictions_first = pd.Series(target_first_train.mean(), index=target_first_valid.index)# Для первой модели
mse = mean_squared_error(target_first_valid, predictions_first)
rmse = mse**0.5#
print("RMSE:", rmse)



RMSE: 44.289591053907365


In [ ]:
predictions_second = pd.Series(target_second_train.mean(), index=target_second_valid.index)# Для второй модели
mse = mean_squared_error(target_second_valid, predictions_second)
rmse = mse**0.5#
print("RMSE:", rmse)


RMSE: 46.02144533725462


In [ ]:
predictions_third = pd.Series(target_third_train.mean(), index=target_third_valid.index)# Для третьей модели
mse = mean_squared_error(target_third_valid, predictions_third)
rmse = mse**0.5#
print("RMSE:", rmse)


RMSE: 44.90234968510566


***Как видно их метрики RMSE, самое малое отклонение от истинных ответов во втором регионе 0.89 баррелей от истинных запасов.
Наибольшее кол-во предсказанных баррелей в третьем регионе.***


## Подготовка к расчёту прибыли

*Все ключевые значения для расчётов сохраним в отдельных переменных.*

In [ ]:
budget = 10000000000# Бюджет разработки месторождений
income_per_barrel = 450000# Доход с одной тысячи баррелей
best_for_development = 200# Число лучших точек для разработки

Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.

In [ ]:
#Общий объем баррелей необходимых для выхода в точку безубыточности
volume = budget/income_per_barrel
volume

22222.222222222223

In [ ]:
#Объем баррелей необходимых для выхода в точку безубыточности, для одной скважины из 200 выбранных
volume_200 = volume/best_for_development
volume_200

111.11111111111111

***Исходя из расчета выход в безубыточности начинается на отметку 22 223 000  баррелей, при этом одна точка добычи, должна давать обьем 111 тысяч беррелей***

## Расчёт прибыли и рисков

In [ ]:
def profit_function(predict,target):
    predict = pd.Series(predict, index=target.index)
    number_of_points = 200
    predict = predict.sort_values(ascending=False)
    product_sum = target[predict.index][:number_of_points].sum()
    return round((product_sum * income_per_barrel) - budget,2)

In [ ]:
print('Прибыль с 200 скважин в первом регионе:',profit_function(predicted_valid_first,target_first_valid))
print('Прибыль с 200 скважин во втором регионе:',profit_function(predicted_valid_second,target_second_valid))
print('Прибыль с 200 скважин в третьем регионе:',profit_function(predicted_valid_third,target_third_valid))

Прибыль с 200 скважин в первом регионе: 3320826043.14
Прибыль с 200 скважин во втором регионе: 2415086696.68
Прибыль с 200 скважин в третьем регионе: 2710349963.6


In [ ]:
state = np.random.RandomState(12345)
def confidence_interval(predict,target):
    samples = []
    predict = pd.Series(predict, index=target.index)
    for i in range(1000):
        predict_sample = predict.sample(n = 500, replace=True, random_state=state)
        target_sample  = target[predict_sample.index]
        samples.append(profit_function(predict_sample,target_sample))
    samples = pd.Series(samples)
    print(f'Средняя прибыль: {samples.mean():,.2f}')
    print(samples.apply(lambda x: x < 0).sum()/len(samples)*100,"%")
    lower = samples.quantile(0.025)
    upper = samples.quantile(0.975)
    return round(lower,2), round(upper,2)


In [ ]:
print('Для первого региона')
print('95% доверительный итервал для Нулевового региона лежит между:',confidence_interval(predicted_valid_first,target_first_valid))



Для первого региона
Средняя прибыль: 425,938,526.91
6.0 %
95% доверительный итервал для Нулевового региона лежит между: (-102090094.84, 947976353.36)


In [ ]:
print('Для второго региона')
print('95% доверительный итервал для Нулевового региона лежит между:',confidence_interval(predicted_valid_second,target_second_valid))

Для второго региона
Средняя прибыль: 518,259,493.70
0.3 %
95% доверительный итервал для Нулевового региона лежит между: (128123231.43, 953612982.06)


In [ ]:
print('Для третьего региона')
print('95% доверительный итервал для Нулевового региона лежит между:',confidence_interval(predicted_valid_third,target_third_valid))

Для третьего региона
Средняя прибыль: 420,194,005.34
6.2 %
95% доверительный итервал для Нулевового региона лежит между: (-115852609.16, 989629939.85)


*** Общий вывод
Исходя из общего анализа данных лучший регион для разработки- регион под номером два.
Имеет самый низкий риск убытков менее 2,5%( фактическое значение 0.3%). Средняя прибыль 518,259,493.70.
Доверительный интервал не лежит в отрицательных значениях.
Выход на точку безубыточности от текущего бюджета на отметке 111-112 тысяч баррелей,с одной скважины.
На втором регионе имеем лучшие показатель обучения модели RMSE=0.89%***